In [1]:
import ast
import base64
import bokeh
import collections
import hashlib
import json
import IPython
import pandas as pd
import math
import matplotlib
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import matplotlib.colors as pltcolors
import networkx as nx
import numpy as np
import os
import re
import scipy
import seaborn as sns
import sys
import unicodedata

from bokeh.io import output_notebook, show
from bokeh.models import LinearAxis, Range1d, HoverTool, PanTool, ResetTool, WheelZoomTool, CrosshairTool, BoxZoomTool
from bokeh.plotting import figure as bokeh_figure
from bokeh.plotting import ColumnDataSource

from bs4 import BeautifulSoup
from matplotlib.backends.backend_pdf import PdfPages
from math import radians, cos, sin, asin, sqrt
from multiprocessing import Pool
from pprint import pprint
# from shapely.geometry import Point, LineString, mapping
from time import time

from scipy.fftpack import fft, ifft, ifft2

from IPython.display import display, display_png, display_html
from IPython.core.display import HTML, Javascript
from IPython.display import Image


IPython.display.display(IPython.display.HTML("<style>.container { width:100%; }</style>"))

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

output_notebook()

Loading BokehJS ...

In [2]:
%%html
<center><img src="https://d397xw3titc834.cloudfront.net/images/original/3/10/310c1e90cdf5e7c22e741ca0fa09d2a3.jpg" width="480" height="360" /></center>

In [3]:
from io import BytesIO

In [4]:
columns = [
    'account',
    'campaign',
    'keyword',
    'impressions',
    'avg_position',
    'impressions_search_rate',
    'quality_level',
    'clicks',
    'avg_cpc',
    'ctr',
    'impressions_keywords_exact_rate',
    'impressions_keywords_lost_rate',
]

In [5]:
def sanify_str(s):
    s = re.sub(r'[\xa0]|[\s+%?]', '', s)
    s = re.sub(',', '.', s)
    mapping = {
        '--': None,
        '<10': '5',
        '>90': '95',
    }
    return mapping.get(s, s)

    
def literal_eval(s, percent=False):
    s = sanify_str(s)
    if s is None:
        return
    s = ast.literal_eval(s)
    return s / 100.0 if percent else s

In [7]:
df_orig = pd.read_csv(
    './ressources/adwords_report_simon.csv',
    sep=',', header=5, names=columns, skipfooter=1,
    engine='python', decimal=',',
    converters={
        'ctr': lambda c: literal_eval(c),
        'quality_level': lambda c: literal_eval(c),
        'avg_cpc': lambda c: literal_eval(c),
        'impressions': lambda c: literal_eval(c),
        'impressions_search_rate': lambda c: literal_eval(c, percent=True),
        'impressions_keywords_exact_rate': lambda c: literal_eval(c, percent=True),
        'impressions_keywords_lost_rate': lambda c: literal_eval(c, percent=True),
    },
    encoding='utf-8'
)

In [81]:
df_orig.head()

,account,campaign,keyword,impressions,avg_position,impressions_search_rate,quality_level,clicks,avg_cpc,ctr,impressions_keywords_exact_rate,impressions_keywords_lost_rate
0,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,+Déménager +Bobigny,2,6.5,0.05,NaN,0,0.0,0.0,0.05,0.6863
1,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,[Déménager Noisy Le Sec],1,6.0,0.25,6.0,0,0.0,0.0,0.25,0.7500
2,production - 69 - 30100020 - running - 397 - 9...,30100020 - 30100020 - SN - B - Activité - Démé...,+Déménageurs +specialiste,4,5.8,0.25,NaN,0,0.0,0.0,0.05,0.5625
3,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,+Déménagement +Levallois,3,6.7,0.05,5.0,0,0.0,0.0,NaN,0.7885
4,production - 69 - 30100020 - running - 397 - 9...,30100020 - 30100020 - SN - C - Prestation - Dé...,"""Prix demenagement""",5,6.0,0.05,8.0,0,0.0,0.0,NaN,0.7812


In [82]:
df = df_orig.copy(deep=True)

In [84]:
pattern_account_product_id = re.compile(r'-\s(?P<product_id>[0-9]+)\s-')


def product_id_from_account(account):
    matches = pattern_account_product_id.search(account)
    if matches is not None:
        return np.int64(matches.groupdict().get('product_id', np.nan))
    return np.nan

In [85]:
df['product_id'] = df.account.apply(product_id_from_account)

In [86]:
df[df.product_id.isnull()]

,account,campaign,keyword,impressions,avg_position,impressions_search_rate,quality_level,clicks,avg_cpc,ctr,impressions_keywords_exact_rate,impressions_keywords_lost_rate,product_id


In [87]:
df.head()

,account,campaign,keyword,impressions,avg_position,impressions_search_rate,quality_level,clicks,avg_cpc,ctr,impressions_keywords_exact_rate,impressions_keywords_lost_rate,product_id
0,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,+Déménager +Bobigny,2,6.5,0.05,NaN,0,0.0,0.0,0.05,0.6863,69
1,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,[Déménager Noisy Le Sec],1,6.0,0.25,6.0,0,0.0,0.0,0.25,0.7500,69
2,production - 69 - 30100020 - running - 397 - 9...,30100020 - 30100020 - SN - B - Activité - Démé...,+Déménageurs +specialiste,4,5.8,0.25,NaN,0,0.0,0.0,0.05,0.5625,69
3,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,+Déménagement +Levallois,3,6.7,0.05,5.0,0,0.0,0.0,NaN,0.7885,69
4,production - 69 - 30100020 - running - 397 - 9...,30100020 - 30100020 - SN - C - Prestation - Dé...,"""Prix demenagement""",5,6.0,0.05,8.0,0,0.0,0.0,NaN,0.7812,69


In [88]:
df.product_id.dtype

dtype('int64')

In [89]:
df.groupby('product_id').clicks.sum().head()

product_id
69      92
106    184
139    113
141    187
189     51
Name: clicks, dtype: int64

In [90]:
df.join(
    df.groupby('product_id').clicks.sum(),
    on='product_id',
    rsuffix='_sum'
).head()

,account,campaign,keyword,impressions,avg_position,impressions_search_rate,quality_level,clicks,avg_cpc,ctr,impressions_keywords_exact_rate,impressions_keywords_lost_rate,product_id,clicks_sum
0,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,+Déménager +Bobigny,2,6.5,0.05,NaN,0,0.0,0.0,0.05,0.6863,69,92
1,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,[Déménager Noisy Le Sec],1,6.0,0.25,6.0,0,0.0,0.0,0.25,0.7500,69,92
2,production - 69 - 30100020 - running - 397 - 9...,30100020 - 30100020 - SN - B - Activité - Démé...,+Déménageurs +specialiste,4,5.8,0.25,NaN,0,0.0,0.0,0.05,0.5625,69,92
3,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,+Déménagement +Levallois,3,6.7,0.05,5.0,0,0.0,0.0,NaN,0.7885,69,92
4,production - 69 - 30100020 - running - 397 - 9...,30100020 - 30100020 - SN - C - Prestation - Dé...,"""Prix demenagement""",5,6.0,0.05,8.0,0,0.0,0.0,NaN,0.7812,69,92


In [91]:
df.groupby(['product_id', 'quality_level']).clicks.sum().head()

product_id  quality_level
69          2.0               0
            3.0               0
            4.0               0
            5.0               3
            6.0              14
Name: clicks, dtype: int64

In [92]:
df.join(
    df.groupby(['product_id', 'quality_level']).clicks.sum(),
    on=['product_id', 'quality_level'],
    rsuffix='_by_quality_sum'
).head(15)

,account,campaign,keyword,impressions,avg_position,impressions_search_rate,quality_level,clicks,avg_cpc,ctr,impressions_keywords_exact_rate,impressions_keywords_lost_rate,product_id,clicks_by_quality_sum
0,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,+Déménager +Bobigny,2,6.5,0.0500,NaN,0,0.00,0.00,0.0500,0.6863,69,NaN
1,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,[Déménager Noisy Le Sec],1,6.0,0.2500,6.0,0,0.00,0.00,0.2500,0.7500,69,14.0
2,production - 69 - 30100020 - running - 397 - 9...,30100020 - 30100020 - SN - B - Activité - Démé...,+Déménageurs +specialiste,4,5.8,0.2500,NaN,0,0.00,0.00,0.0500,0.5625,69,NaN
3,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,+Déménagement +Levallois,3,6.7,0.0500,5.0,0,0.00,0.00,NaN,0.7885,69,3.0
4,production - 69 - 30100020 - running - 397 - 9...,30100020 - 30100020 - SN - C - Prestation - Dé...,"""Prix demenagement""",5,6.0,0.0500,8.0,0,0.00,0.00,NaN,0.7812,69,37.0
5,production - 69 - 30100020 - running - 397 - 9...,30100020 - 30100020 - SN - C - Prestation - Dé...,"""Prix déménagements""",4,5.5,0.1429,NaN,0,0.00,0.00,NaN,0.6429,69,NaN
6,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,"""Déménagement Bourg La Reine""",2,5.0,0.0500,2.0,0,0.00,0.00,NaN,0.5385,69,0.0
7,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,[Déménagement Groslay],3,4.3,0.3333,6.0,0,0.00,0.00,0.3333,0.6667,69,14.0
8,production - 69 - 30100020 - running - 397 - 9...,30100020 - 30100020 - SN - C - Prestation - Dé...,+demenagements +Prix,1,4.0,0.1667,NaN,0,0.00,0.00,NaN,0.8333,69,NaN
9,production - 69 - 30100020 - running - 397 - 9...,30100020 - 30100020 - SN - B - Activité - Démé...,+Déménageurs +Service,1,7.0,0.0500,NaN,0,0.00,0.00,NaN,0.8000,69,NaN


In [93]:
pd.DataFrame(df.groupby(['product_id', 'quality_level']).clicks.sum()).head(10)

clicks
product_id quality_level        
69         2.0                 0
           3.0                 0
           4.0                 0
           5.0                 3
           6.0                14
           7.0                13
           8.0                37
           9.0                 1
           10.0               18
106        1.0                 0

In [98]:
class Builder:
    """Builder class

    Ability to,
        - build Dataframe from specific csv files (testing multiple encodings)
    """

    encodings = []

    def read_csv(self, filename, encodings=None, **reader_options):
        encodings = encodings or self.encodings
        for encoding in encodings:
            try:
                df = pd.read_csv(filename, encoding=encoding, **reader_options)
            except UnicodeDecodeError:
                continue
            except Exception as e:
                raise e
            break
        return df

In [124]:
def sanify_str(s):
    s = re.sub(r'[\xa0]|[\s+%?]', '', s)
    s = re.sub(',', '.', s)
    mapping = {
        '--': None,
        '<10': '5',
        '>90': '95',
    }
    return mapping.get(s, s)

    
def literal_eval(s, percent=False):
    s = sanify_str(s)
    if s is None:
        return
    s = ast.literal_eval(s)
    return s / 100.0 if percent else s


class AdwordsReportBuilder(Builder):
    
    columns = [
        'account',
        'campaign',
        'keyword',
        'impressions',
        'avg_position',
        'impressions_search_rate',
        'quality_level',
        'clicks',
        'avg_cpc',
        'ctr',
        'impressions_keywords_exact_rate',
        'impressions_keywords_lost_rate',
    ]
    encodings = [
        None,
        'UTF-8',
        'cp1252',
    ]
    patterns = {
        'product_id': re.compile(r'-\s(?P<product_id>[0-9]+)\s-'),
    }
    reader_options = {
        'sep': ',',
        'header': 5,
        'names': columns,
        'skipfooter': 1,
        'engine': 'python',
        'decimal': ',',
    }
    
    def __init__(self, filename, encodings=None, **reader_options):
        self.filename = filename
        self.reader_options.update({
            'converters': {
                'ctr': lambda c: literal_eval(c),
                'quality_level': lambda c: literal_eval(c),
                'avg_cpc': lambda c: literal_eval(c),
                'impressions': lambda c: literal_eval(c),
                'impressions_search_rate': lambda c: literal_eval(c, percent=True),
                'impressions_keywords_exact_rate': lambda c: literal_eval(c, percent=True),
                'impressions_keywords_lost_rate': lambda c: literal_eval(c, percent=True),
            },
        })
        self.reader_options.update(reader_options)
        
    def build(self):
        self.df = self.read_csv(self.filename, **self.reader_options)
        self.df['product_id'] = self.df.account.apply(self.product_id_from_account)
        return self.df

    def product_id_from_account(self, account):
        matches = self.patterns['product_id'].search(account)
        if matches is not None:
            return np.int64(matches.groupdict().get('product_id', np.nan))
        return np.nan
    
    def aggregates(self, target, cols, agg, suffix=None):
        suffix = suffix or '_{agg}'.format(agg=agg)
        self.df = self.df.join(
            self.df.groupby(cols)[target].aggregate(agg),
            on=cols,
            rsuffix=suffix
        )

In [119]:
builder = AdwordsReportBuilder('./ressources/adwords_report_simon.csv')

In [120]:
df = builder.build()

In [121]:
df.head()

,account,campaign,keyword,impressions,avg_position,impressions_search_rate,quality_level,clicks,avg_cpc,ctr,impressions_keywords_exact_rate,impressions_keywords_lost_rate,product_id
0,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,+Déménager +Bobigny,2,6.5,0.05,NaN,0,0.0,0.0,0.05,0.6863,69
1,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,[Déménager Noisy Le Sec],1,6.0,0.25,6.0,0,0.0,0.0,0.25,0.7500,69
2,production - 69 - 30100020 - running - 397 - 9...,30100020 - 30100020 - SN - B - Activité - Démé...,+Déménageurs +specialiste,4,5.8,0.25,NaN,0,0.0,0.0,0.05,0.5625,69
3,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,+Déménagement +Levallois,3,6.7,0.05,5.0,0,0.0,0.0,NaN,0.7885,69
4,production - 69 - 30100020 - running - 397 - 9...,30100020 - 30100020 - SN - C - Prestation - Dé...,"""Prix demenagement""",5,6.0,0.05,8.0,0,0.0,0.0,NaN,0.7812,69


In [122]:
builder.aggregates('clicks', ['product_id', 'quality_level'], 'sum')

In [123]:
builder.df.head()

,account,campaign,keyword,impressions,avg_position,impressions_search_rate,quality_level,clicks,avg_cpc,ctr,impressions_keywords_exact_rate,impressions_keywords_lost_rate,product_id,clicks_sum
0,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,+Déménager +Bobigny,2,6.5,0.05,NaN,0,0.0,0.0,0.05,0.6863,69,NaN
1,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,[Déménager Noisy Le Sec],1,6.0,0.25,6.0,0,0.0,0.0,0.25,0.7500,69,14.0
2,production - 69 - 30100020 - running - 397 - 9...,30100020 - 30100020 - SN - B - Activité - Démé...,+Déménageurs +specialiste,4,5.8,0.25,NaN,0,0.0,0.0,0.05,0.5625,69,NaN
3,production - 69 - 30100020 - running - 397 - 9...,30100020 - SN - x - Déménagements - Geozone_ma...,+Déménagement +Levallois,3,6.7,0.05,5.0,0,0.0,0.0,NaN,0.7885,69,3.0
4,production - 69 - 30100020 - running - 397 - 9...,30100020 - 30100020 - SN - C - Prestation - Dé...,"""Prix demenagement""",5,6.0,0.05,8.0,0,0.0,0.0,NaN,0.7812,69,37.0
